In [0]:
#copy datasets from drive
%cp '/content/drive/My Drive/SIMS_Datsaet/test.txt' /content/
%cp '/content/drive/My Drive/SIMS_Datsaet/training.txt' /content/
%cp '/content/drive/My Drive/SIMS_Datsaet/validation.txt' /content/
%cp '/content/drive/My Drive/SIMS_Datsaet/images.zip' /content/
%cp '/content/drive/My Drive/SIMS_Datsaet/Annotations_in_3_formats.zip' /content/

In [0]:
#unzip them
!unzip /content/Annotations_in_3_formats.zip
!unzip /content/images.zip

In [0]:
#copy all the annotations from pascal voc to images folder
%cp -a /content/annotations/pascal_voc/Annotations/. /content/images/

In [0]:
!mkdir Dataset

!mkdir Dataset/train
!mkdir Dataset/train/images
!mkdir Dataset/train/annotations

!mkdir Dataset/valid
!mkdir Dataset/valid/images
!mkdir Dataset/valid/annotations

!mkdir Dataset/test
!mkdir Dataset/test/images
!mkdir Dataset/test/annotations

In [0]:
#preprocess: copy corresponding annotations in {train,test,val} as per paths in {datasets}

import shutil
import os
#copy files based on {datasets in txt}

def copy(filepath,dest_dir):
    with open(filepath) as fp:
        for line in fp:
            os.makedirs(dest_dir, exist_ok=True)  # succeeds even if directory exists.
            shutil.copy(line.replace('\n', ''), dest_dir+"/images") #image
            shutil.copy(line.replace('.jpg\n', '.xml'), dest_dir+"/annotations") ##label

#copy files based on {datasets in txt}
copy('training.txt','Dataset/train/')
copy('validation.txt','Dataset/valid/')
copy('test.txt','Dataset/test/')

In [0]:
#from preprocessing.pascal_voc_to_csv import xml_to_csv

import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path,image_path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        #print(xml_file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bbx = member.find('bndbox')
            xmin = int(bbx.find('xmin').text)
            ymin = int(bbx.find('ymin').text)
            xmax = int(bbx.find('xmax').text)
            ymax = int(bbx.find('ymax').text)
            label = member.find('name').text
            
            size = root.find('size') #contains height and width of image
            w = int(size.find('width').text)
            h = int(size.find('height').text)
            
            # truncate any bb coordinates that fall outside of the image
            xmin = max(0, xmin)
            ymin = max(0, ymin)
            xmax = min(w, xmax)
            ymax = min(h, ymax)
            
            # # ignore the bounding boxes where the minimum values are larger
            # # than the maximum values and vice-versa due to annotation errors
            if xmin >= xmax or ymin >= ymax:
                continue
            elif xmax <= xmin or ymax <= ymin:
                continue
            
            file_path = image_path+root.find('filename').text+".jpg"
            value = (file_path,
                     xmin,
                     ymin,
                     xmax,
                     ymax,
                     label,
                     )
            xml_list.append(value)
    column_name = ['filename','xmin', 'ymin', 'xmax', 'ymax','class']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    root_path = "Dataset/"
    datasets = ['train','test','valid']
    for ds in datasets:
        #image_path = os.path.join(os.getcwd(), root_path+ds, 'annotations')
        image_path = os.path.join(root_path+ds, 'annotations/')
        print(image_path)
        update_path = os.path.join(root_path+ds, 'images/')
        xml_df = xml_to_csv(image_path,update_path)
        xml_df.to_csv('labels_{}.csv'.format(ds), index=None, header=False)
        print('Successfully converted xml to csv.')


main()

Dataset/train/annotations/
Successfully converted xml to csv.
Dataset/test/annotations/
Successfully converted xml to csv.
Dataset/valid/annotations/
Successfully converted xml to csv.


In [0]:
import pandas as pd
csv_df = pd.read_csv('/content/labels_train.csv',header=0)
csv_df

,Dataset/train/images/2651.jpg,80,153,241,367,trainer
0,Dataset/train/images/2651.jpg,188,347,328,523,trainer
1,Dataset/train/images/2651.jpg,441,60,602,276,trainer
2,Dataset/train/images/2651.jpg,431,218,588,411,trainer
3,Dataset/train/images/2651.jpg,469,330,796,704,chartered
4,Dataset/train/images/0163.jpg,359,202,603,382,airliner
...,...,...,...,...,...,...
23871,Dataset/train/images/0918.jpg,220,374,278,471,stairtruck
23872,Dataset/train/images/0918.jpg,357,676,503,732,bus
23873,Dataset/train/images/0918.jpg,562,81,650,132,stairtruck
23874,Dataset/train/images/0918.jpg,485,428,514,492,other


In [0]:
#Convert csv into txt by changing extension manually